# 🎮 Keep the Land! ML-Agents Training on Google Colab

## Paper.io 스타일 영역 확보 게임 프로젝트

**프로젝트 개요:** Unity ML-Agents를 활용한 Keep the Land! 게임에서 4개의 AI 에이전트가 PPO 강화학습을 통해 최적의 전략을 학습하는 프로젝트입니다.

---

### 📋 이 노트북의 구성
1. **환경 설정** - Python 버전 확인 및 ML-Agents 설치
2. **프로젝트 준비** - GitHub에서 Keep the Land! 프로젝트 클론
3. **Unity 빌드 업로드** - 헤드리스 빌드 파일 준비
4. **학습 설정** - 하이퍼파라미터 구성
5. **학습 실행** - TensorBoard와 함께 학습 진행
6. **모델 다운로드** - 학습된 .onnx 파일 저장

### ⚡ 시작하기 전에
- **GPU 런타임** 설정: 런타임 → 런타임 유형 변경 → GPU (T4) 선택
- **Unity 헤드리스 빌드** 파일이 필요합니다 (Linux 플랫폼)
- 예상 학습 시간: **30분 ~ 2시간** (설정에 따라)

## 🔧 Step 1: 환경 설정 및 Python 버전 확인

In [ ]:
import sys
import os

print("🐍 Python 버전 확인:")
print(f"Version: {sys.version}")
print(f"Platform: {sys.platform}")

if sys.version_info[0] < 3:
    raise Exception("❌ ERROR: ML-Agents는 Python 3가 필요합니다!")
else:
    print("✅ Python 3 확인됨!")

# GPU 확인
print("\n🖥️ GPU 상태 확인:")
!nvidia-smi -L

## 📦 Step 2: ML-Agents 설치

In [ ]:
print("📥 ML-Agents 설치 중...")

# ML-Agents 최신 버전 설치
!pip install mlagents==0.30.0 -q
!pip install torch torchvision torchaudio -q

print("✅ ML-Agents 설치 완료!")

# 설치 확인
!mlagents-learn --help | head -10

## 📂 Step 3: Keep the Land! ML-Agents 프로젝트 준비

In [ ]:
import os

print("📂 Keep the Land! ML-Agents 프로젝트 준비...")

# 작업 디렉토리로 이동
os.chdir('/content')

# 만약 이미 폴더가 있다면 제거
!rm -rf Project-AI-Paper.io-ML-Agents

# GitHub에서 프로젝트 클론 (실제 프로젝트 URL로 변경 필요)
!git clone https://github.com/your-username/Project-AI-Paper.io-ML-Agents.git

# 프로젝트 폴더로 이동
os.chdir('/content/Project-AI-Paper.io-ML-Agents')

print("✅ 프로젝트 클론 완료!")
print("\n📁 프로젝트 구조:")
!ls -la

## 🏗️ Step 4: Unity 헤드리스 빌드 업로드

### ⚠️ 중요한 단계!

이 단계에서는 Unity에서 **Linux 헤드리스 빌드**를 업로드해야 합니다.

#### 로컬 Unity에서 빌드 생성 방법:
1. Unity 프로젝트 열기
2. **File → Build Settings**
3. **Platform: Linux** 선택
4. **Server Build** 체크박스 선택
5. **Player Settings** → **Headless Mode** 활성화
6. 빌드 이름: `PaperIO_JM_Linux`
7. **Build** 클릭

#### 빌드 파일 업로드:

In [ ]:
from google.colab import files
import zipfile
import os

print("📤 Unity 헤드리스 빌드 업로드")
print("\n🎯 업로드할 파일들:")
print("- PaperIO_JM_Linux.x86_64 (실행 파일)")
print("- PaperIO_JM_Linux_Data/ (폴더 전체)")
print("- UnityPlayer.so")
print("\n📝 tip: 전체를 zip으로 압축해서 업로드하면 편합니다!")

# 파일 업로드
uploaded = files.upload()

# zip 파일이면 압축 해제
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"\n📦 {filename} 압축 해제 중...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('./build/')
        os.remove(filename)
        print("✅ 압축 해제 완료!")

# 실행 권한 부여
print("\n🔐 실행 권한 설정...")
!find ./build -name "*.x86_64" -exec chmod +x {} \;
!find ./build -name "*.so" -exec chmod +x {} \;

print("\n📁 업로드된 빌드 파일:")
!ls -la ./build/

## ⚙️ Step 5: 학습 설정 (하이퍼파라미터)

In [ ]:
# 환경 설정
ENV_NAME = "./build/PaperIO_JM_Linux.x86_64"  # 빌드 파일 경로
RUN_ID = "paperio_colab_run_001"  # 학습 실행 ID
MAX_STEPS = 5000000  # 최대 학습 스텝 (500만)

print(f"🎮 환경 이름: {ENV_NAME}")
print(f"🏷️ 실행 ID: {RUN_ID}")
print(f"📈 최대 스텝: {MAX_STEPS:,}")

# Paper.io 전용 학습 설정 파일 생성
config_content = f"""behaviors:
  MyAgent:
    trainer_type: ppo
    hyperparameters:
      batch_size: 2048
      buffer_size: 20480
      learning_rate: 3.0e-4
      beta: 5.0e-3
      epsilon: 0.2
      lambd: 0.95
      num_epoch: 3
      learning_rate_schedule: linear
    network_settings:
      normalize: false
      hidden_units: 512
      num_layers: 3
      vis_encode_type: simple
    reward_signals:
      extrinsic:
        gamma: 0.99
        strength: 1.0
    max_steps: {MAX_STEPS}
    time_horizon: 1000
    summary_freq: 10000
    checkpoint_interval: 50000
    keep_checkpoints: 10
"""

# 설정 파일 저장
with open('paperio_config.yaml', 'w') as f:
    f.write(config_content)

print("\n✅ Paper.io 전용 설정 파일 생성 완료!")
print("\n📄 설정 내용:")
!cat paperio_config.yaml

## 📊 Step 6: TensorBoard 설정

In [ ]:
# TensorBoard 확장 로드
%load_ext tensorboard

# TensorBoard 시작 (백그라운드에서 실행)
%tensorboard --logdir results --port 6006

print("📊 TensorBoard가 시작되었습니다!")
print("학습이 시작되면 위 창에서 실시간 진행 상황을 확인할 수 있습니다.")

## 🚀 Step 7: Keep the Land! AI 학습 시작!

### 🎯 학습 목표
- **영역 확장**: AI가 효율적으로 영역을 넓히는 전략 학습
- **생존 전략**: 상대방의 공격을 피하고 안전하게 복귀
- **공격 전략**: 상대방의 궤적을 끊어 점수 획득
- **위험 회피**: 벽 충돌이나 자살 방지

### ⏱️ 예상 소요 시간
- **초기 학습**: 10-20분 (기본적인 움직임)
- **중급 학습**: 30-60분 (영역 확장 학습)
- **고급 학습**: 1-2시간 (전략적 플레이)

In [ ]:
import time
import subprocess
from datetime import datetime

print("🎮 Keep the Land! ML-Agents 학습 시작!")
print(f"⏰ 시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*60)

# 학습 시작
try:
    # 실제 학습 명령어 실행
    process = subprocess.run([
        "mlagents-learn",
        "paperio_config.yaml",
        f"--run-id={RUN_ID}",
        f"--env={ENV_NAME}",
        "--no-graphics",
        "--force"
    ], 
    cwd="/content/Project-AI-Paper.io-ML-Agents",
    capture_output=False,
    text=True)
    
    print(f"\n🎉 학습 완료! (종료 코드: {process.returncode})")
    print(f"⏰ 종료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
except KeyboardInterrupt:
    print("\n⏹️ 사용자에 의해 학습이 중단되었습니다.")
except Exception as e:
    print(f"\n❌ 오류 발생: {str(e)}")
    print("\n🔧 문제 해결 방법:")
    print("1. Unity 빌드 파일 경로 확인")
    print("2. 빌드 파일 실행 권한 확인")
    print("3. GPU 메모리 확인")
    
print("\n📁 학습 결과 폴더:")
!ls -la results/

## 📈 Step 8: 학습 결과 확인

In [ ]:
import os
import glob

print("📊 Keep the Land! AI 학습 결과 분석")
print("="*50)

# 결과 폴더 확인
results_path = f"results/{RUN_ID}"

if os.path.exists(results_path):
    print(f"✅ 학습 결과 폴더 발견: {results_path}")
    
    # ONNX 모델 파일 확인
    onnx_files = glob.glob(f"{results_path}/*/*.onnx")
    if onnx_files:
        print(f"\n🧠 학습된 AI 모델: {len(onnx_files)}개")
        for onnx_file in onnx_files:
            file_size = os.path.getsize(onnx_file) / 1024  # KB
            print(f"  📄 {onnx_file} ({file_size:.1f}KB)")
    
    # 체크포인트 확인
    checkpoint_files = glob.glob(f"{results_path}/*/*.pt")
    if checkpoint_files:
        print(f"\n💾 체크포인트: {len(checkpoint_files)}개")
        
    # 로그 파일 확인
    print("\n📝 학습 폴더 구조:")
    !find $results_path -type f | head -20
    
else:
    print("❌ 학습 결과 폴더를 찾을 수 없습니다.")
    print("전체 results 폴더 확인:")
    !ls -la results/

## 💾 Step 9: 학습된 AI 모델 다운로드

In [ ]:
from google.colab import files
import shutil
import zipfile
import glob
import os

print("💾 Keep the Land! AI 모델 다운로드 준비")
print("="*40)

# ONNX 모델 파일 찾기
onnx_files = glob.glob(f"results/{RUN_ID}/*/*.onnx")

if onnx_files:
    print(f"✅ {len(onnx_files)}개의 AI 모델 발견!")
    
    # 다운로드 폴더 생성
    download_folder = "keep_the_land_ai_models"
    os.makedirs(download_folder, exist_ok=True)
    
    # 모델 파일들을 다운로드 폴더로 복사
    for i, onnx_file in enumerate(onnx_files):
        model_name = f"KeepTheLand_AI_Model_{i+1}.onnx"
        shutil.copy2(onnx_file, f"{download_folder}/{model_name}")
        print(f"📄 {model_name} 준비 완료")
    
    # 설정 파일도 포함
    shutil.copy2("paperio_config.yaml", f"{download_folder}/training_config.yaml")
    
    # README 파일 생성
    readme_content = f"""# Keep the Land! ML-Agents 학습 결과

## 📁 파일 설명
- `KeepTheLand_AI_Model_*.onnx`: 학습된 AI 모델 파일
- `training_config.yaml`: 학습에 사용된 설정

## 🎮 Unity에서 사용 방법
1. Unity 프로젝트에서 MyAgent GameObject 선택
2. Behavior Parameters 컴포넌트의 Model 필드에 .onnx 파일 드래그
3. Behavior Type을 'Default'로 설정
4. Play 버튼 클릭하여 AI 플레이 확인

## 📊 학습 정보
- 학습 ID: {RUN_ID}
- 최대 스텝: {MAX_STEPS:,}
- 학습 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
- 플랫폼: Google Colab (GPU)

## 🏆 Keep the Land! 프로젝트
Unity ML-Agents를 활용한 영역 확보 게임 AI 개발
"""
    
    with open(f"{download_folder}/README.md", "w", encoding="utf-8") as f:
        f.write(readme_content)
    
    # ZIP 파일로 압축
    zip_filename = f"KeepTheLand_AI_Models_{RUN_ID}.zip"
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(download_folder):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, download_folder)
                zipf.write(file_path, arcname)
    
    print(f"\n📦 {zip_filename} 생성 완료!")
    print(f"📊 파일 크기: {os.path.getsize(zip_filename) / 1024:.1f}KB")
    
    # 다운로드 시작
    print("\n⬇️ 다운로드 시작...")
    files.download(zip_filename)
    
    print("\n🎉 다운로드 완료!")
    print("Unity 프로젝트에서 .onnx 파일을 사용하여 AI를 테스트해보세요!")
    
else:
    print("❌ ONNX 모델 파일을 찾을 수 없습니다.")
    print("학습이 완료되었는지 확인해주세요.")

## 🎯 학습 완료 요약

### ✅ 완료된 작업
1. **환경 설정**: Python 3 + ML-Agents 설치
2. **프로젝트 준비**: Keep the Land! ML-Agents 프로젝트 클론
3. **Unity 빌드**: 헤드리스 빌드 업로드 및 권한 설정
4. **학습 설정**: PPO 알고리즘 + 최적화된 하이퍼파라미터
5. **AI 학습**: 4개 에이전트의 강화학습 진행
6. **모델 저장**: ONNX 형식으로 학습된 AI 모델 생성

### 🧠 AI가 학습한 전략들
- **영역 확장**: 효율적인 경로로 최대 영역 확보
- **생존 전략**: 위험 상황에서의 안전한 복귀
- **공격 전략**: 상대방 궤적을 끊어 점수 획득
- **위험 회피**: 벽 충돌이나 자기 궤적 충돌 방지

### 🚀 다음 단계
1. **Unity 테스트**: 다운로드한 .onnx 모델을 Unity에서 테스트
2. **성능 평가**: AI의 플레이 스타일과 승률 분석
3. **하이퍼파라미터 조정**: 더 나은 성능을 위한 재학습
4. **다양한 시나리오**: 다른 맵이나 규칙으로 추가 학습

---

### 🏆 Keep the Land! 프로젝트
**영역 확보 게임 AI 개발**: Unity + ML-Agents + PPO 강화학습을 활용한 지능형 게임 AI 개발

**🎮 프로젝트 특징**:
- 실시간 4인 멀티플레이어 AI
- 8차원 관찰 공간으로 효율적인 상황 인식
- 영역 확장, 생존, 공격의 균형잡힌 전략
- Google Colab GPU를 활용한 클라우드 학습

**💡 기술 스택**: Unity 2022.3 LTS, ML-Agents 2.0, PPO Algorithm, Python 3.8+, TensorBoard